In [1]:
import os
from pprint import pprint
from IPython.display import Markdown, display
import requests
from requests.models import Response
from bs4 import BeautifulSoup

In [2]:
def _msg(role, content): #returns a dict
    return {'role':role, 'content':content}

def user(content):
    return _msg('user', content)

def system(content):
    return _msg('system', content)

def assistant(content):
    return _msg('assistant', content)

def payload(*,MODEL,chat_history, stream):
    return {
        "model": MODEL,
        "messages": chat_history,
        "stream": stream
    }

# Caching

In [3]:
from diskcache import Cache
# cache = Cache() --> this is a temporary cache where it's not saved when the program is restart
cache = Cache(directory = '.ollama_websummerizer_caching') # This is not a temporary where tha cache is saved in a directory
                                                # and can be accessed any time


In [4]:
from hashlib import md5
import json
# print(md5(b'dana the one').hexdigest())
# md5 works with only str

def cache_key(key, **kwargs):
    kwargs_str = json.dumps(kwargs, sort_keys = True)
    kwargs_hashed = md5(kwargs_str.encode('utf-8')).hexdigest()
    hashed_key = f'{key}__{kwargs_hashed}'
    return hashed_key

def _ollama_request_cache_key(*, OLLAMA_API, json, headers, **kwargs): # the first * to make the function only accepts key words args
    return cache_key('ollama_request', OLLAMA_API = OLLAMA_API, json = json, headers = headers, **kwargs)

In [5]:
def response_type(data_dict, status_code = 200):
    response = Response()
    response.status_code = status_code
    response._content = json.dumps(data_dict).encode('utf-8')
    response.headers['Content-Type'] = 'application/json'
    return response

In [6]:
from openai.types.chat import ChatCompletion
from functools import update_wrapper
CACHE_MISS_SENTINEL = object() # we need to create sth that can be created ONLY once and can't be created accidentally anytime, like a memory address => an object

def cached_ollama_post_request(*, OLLAMA_API, json, headers, **kwargs) -> ChatCompletion : # the 'async' because we want to use async functions
    # MAKE KEY
    cache_key = _ollama_request_cache_key(OLLAMA_API = OLLAMA_API, json = json, headers = headers, **kwargs)
    cached_value = cache.get(cache_key, default = CACHE_MISS_SENTINEL) # we will use this as default when the key can't be found in the cache
    
    # CACHE MISS
    if cached_value is CACHE_MISS_SENTINEL:
        response = requests.post(OLLAMA_API, json=json, headers=headers, **kwargs) # a) make post request
        cache.set(cache_key, response.json()) # b) set the output in the cache
        return response # this is a chat completion type

    # CACHE HIT
    else:
        return response_type(cached_value) # Cached Value is the vlue we need

cached_chat_completion = update_wrapper(cached_ollama_post_request, requests.post) # for auto completion

# Practice

In [7]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [8]:
chat_history = [
    user('Describe some of the business applications of Generative AI')
]
pprint(chat_history)

[{'content': 'Describe some of the business applications of Generative AI',
  'role': 'user'}]


In [9]:
pl = payload(MODEL = MODEL, chat_history = chat_history, stream = False)

In [10]:
response = cached_ollama_post_request(OLLAMA_API =OLLAMA_API, json=pl, headers=HEADERS)

In [11]:
type(response)

requests.models.Response

In [12]:
response.json()

{'model': 'llama3.2',
 'created_at': '2025-05-19T12:46:35.6744945Z',
 'message': {'role': 'assistant',
  'content': 'Generative AI has numerous business applications across various industries, including:\n\n1. **Content Creation**: Generative AI can generate high-quality content such as articles, social media posts, product descriptions, and more. This helps businesses save time and resources on content creation, while also providing a consistent tone and style.\n\n2. **Customer Service Chatbots**: Generative AI-powered chatbots can understand and respond to customer queries in real-time, improving customer service experience and reducing the workload of human customer support agents.\n\n3. **Marketing Automation**: Generative AI can help automate marketing campaigns by generating personalized product recommendations, email subject lines, and ad copy, leading to increased engagement and conversion rates.\n\n4. **Product Design and Development**: Generative AI can be used to design new 

In [13]:
cached_response = cached_ollama_post_request(OLLAMA_API =OLLAMA_API, json=pl, headers=HEADERS)

In [14]:
type(cached_response)

requests.models.Response

In [15]:
cached_response.json() == response.json()

True

In [16]:
Markdown(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as articles, social media posts, product descriptions, and more. This helps businesses save time and resources on content creation, while also providing a consistent tone and style.

2. **Customer Service Chatbots**: Generative AI-powered chatbots can understand and respond to customer queries in real-time, improving customer service experience and reducing the workload of human customer support agents.

3. **Marketing Automation**: Generative AI can help automate marketing campaigns by generating personalized product recommendations, email subject lines, and ad copy, leading to increased engagement and conversion rates.

4. **Product Design and Development**: Generative AI can be used to design new products, generate 3D models, and even create prototypes, reducing the time and cost associated with traditional product development methods.

5. **Financial Analysis and Predictive Modeling**: Generative AI can analyze large datasets to identify patterns and trends, providing valuable insights for businesses to make informed investment decisions and predict future market movements.

6. **Image and Video Editing**: Generative AI can edit images and videos in real-time, allowing for faster and more efficient post-production workflows, reducing costs associated with human editors.

7. **Recommendation Systems**: Generative AI-powered recommendation systems can suggest products or services to customers based on their past behavior and preferences, increasing the likelihood of a sale.

8. **Cybersecurity Threat Analysis**: Generative AI can analyze network traffic patterns to identify potential security threats in real-time, helping businesses detect and respond to cyber-attacks more effectively.

9. **Language Translation**: Generative AI-powered language translation tools can translate text and speech in real-time, improving communication between businesses and international customers or partners.

10. **Supply Chain Optimization**: Generative AI can analyze supply chain data to identify bottlenecks and opportunities for improvement, helping businesses optimize their logistics and reduce costs.

11. **Predictive Maintenance**: Generative AI can analyze sensor data from machines to predict when maintenance is required, reducing downtime and increasing overall equipment effectiveness.

12. **Data Annotation and Labeling**: Generative AI can automatically annotate and label data, reducing the time and cost associated with manual data labeling tasks.

13. **Speech Recognition and Transcription**: Generative AI-powered speech recognition tools can transcribe spoken language into text in real-time, improving communication between businesses and customers.

14. **Creative Writing and Storytelling**: Generative AI can generate creative content such as stories, poems, or even entire scripts, helping businesses tap into the power of artificial creativity.

15. **Music and Audio Generation**: Generative AI can create original music tracks or audio elements in real-time, opening up new opportunities for businesses to create engaging multimedia experiences.

These are just a few examples of the many business applications of Generative AI. As the technology continues to evolve, we can expect to see even more innovative uses across various industries.

# The Project - Web Summerizer

### Web Object - beautiful soup

In [17]:
class Website:
    url: str
    title: str
    content: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "Title not found"

        for irrelevant in soup.body(['script','style','img', 'input']):
            irrelevant.decompose() # to remove these tags from the body

        self.content = soup.body.get_text(separator = '\n', strip = True)

### Ollama call

In [18]:
def user_prompt_for(website):
    prompt = f'You are looking for a website titled: \"{website.title}\"\n\n'
    prompt += f'the content of this website is as follows:\n{website.content}\n\n'
    prompt += 'Provide a summary for this website in markdown'
    return user(prompt)

In [19]:
chat_history_WS = [
    system('You are an assitant tool that analyzes a website content,\
                        and provides a summery. Ignore text that might be navigation related.\
                        Respond in Markdown.')
]

In [20]:
chat_history_WS.append(user_prompt_for(Website('https://www.lookmovie2.to/movies/page/6')))

In [21]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

payload_WS = payload(MODEL = MODEL, chat_history = chat_history_WS, stream = False)

In [22]:
response = cached_ollama_post_request(OLLAMA_API = OLLAMA_API, json=payload_WS, headers=HEADERS)

In [23]:
Markdown(response.json()['message']['content'])

# Website Summary: WatchlatestMovies2

## Overview

Watchlatestmovies2 is a free movie and TV shows streaming site with over 50,000 titles to choose from. No registration or payment is required to watch content online.

## Features

* Over 50,000 movies and TV shows available for streaming
* No registration or payment required
* Ability to bookmark and share full episodes for later viewing
* Contact us page for support and feedback

## User Experience

The website features a simple navigation menu with sections for movies and TV shows. The content is displayed in a list format, with each title including its rating, release year, and genre.

## Technical Details

* Website URL: [not provided]
* Page structure: Simple navigation menu with list-based content display
* User interface: Basic text-based interface

## Potential Security Concerns

* Lack of registration or payment required may indicate a potential security risk
* Contact us page appears to be functional but may not provide adequate support for users

In [24]:
chat_history_WS.append(assistant(response.json()['message']['content']))
chat_history_WS.append(user('summerize all the movies and their ratings from the heighest to the lowest. Notice that every movie has its ratings above its name'))

In [25]:
payload_WS2 = payload(MODEL = MODEL, chat_history = chat_history_WS, stream = False)
response = cached_ollama_post_request(OLLAMA_API = OLLAMA_API, json=payload_WS2, headers=HEADERS)

In [26]:
Markdown(response.json()['message']['content'])

# Movies Summary: WatchlatestMovies2

## Ranked Movies by Rating (Highest to Lowest)

1. **8.4 / 10** - TinMan (2024, FHD)
2. **7.6 / 10** - Maharaja (2020, FHD)
3. **7.3 / 10** - Snow White (2024, FHD)
4. **7.1 / 10** - How Deep Is the Ocean (2023, FHD)
5. **7.0 / 10** - Artie Shaw: Time Is All You've Got (1985, FHD)
6. **7.0 / 10** - The Devil Comes to Kansas City (2022, FHD)
7. **6.9 / 10** - Love in the Clouds (2025, FHD)
8. **6.8 / 10** - Catalyst (2024, FHD)
9. **6.7 / 10** - Britain and the Blitz (2025, FHD)
10. **6.6 / 10** - Steve Rannazzisi: Manchild (2013, FHD)
11. **6.6 / 10** - The Half-Breed (1916, FHD)
12. **6.5 / 10** - The Accidental Getaway Driver (2025, FHD)
13. **6.4 / 10** - Mahjong King (2022, HD) [ missing from original content]
14. **6.2 / 10** - Maybe Forever (2024, FHD)
15. **6.2 / 10** - Roy Orbison Forever (2023, FHD)
16. **6.1 / 10** - Artie Shaw: Time Is All You've Got (1985, FHD) [ duplicate entry]
17. **6.0 / 10** - Vini Jr. (2023, FHD)
18. **6.0 / 10** - Sentimentaaal (2025, FHD)
19. **6.0 / 10** - How Deep Is the Ocean (2024, FHD) [ duplicate entry]
20. **5.9 / 10** - A Minecraft Movie (2023, FHD)

... and many more movies with lower ratings.

## Note:

The list above only includes movies mentioned in the original content and may not be a comprehensive list of all available movies on WatchlatestMovies2.

# Other Approches

### 1. Ollama Package

In [27]:
import ollama
# response = ollama.chat(model = MODEL, messages = chat_history)
# print(response['message']['content'])

### 2. OpenAI ChatCompletions endpoint

In [28]:
from openai import OpenAI

ollama_via_openai = OpenAI(base_url = 'http://localhost:11434/v1', api_key = 'ollama')
# response = ollama_via_openai.chat.completions.create(model = MODEL, messages = chat_history)
# print(respnse.choices[0].message.content)

### 3. OpenAI ChatCompletions endpoint with DeepSeek model

In [29]:
!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest 
pulling aabd4debf0c8... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 GB                         
pulling 369ca498f347... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  387 B                         
pulling 6e4c38e1172f... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 KB                         
pulling f4d24e9138dd... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  148 B                         
pulling a85fe2a2e58e... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [30]:
response = ollama_via_openai.chat.completions.create(model="deepseek-r1:1.5b", messages=chat_history)
print(response.choices[0].message.content)

<think>
Okay, so I'm trying to understand the business applications of generative AI. From what I've heard before, generative AI is great at creating things like text, images, and even music. But how exactly can businesses use this? Let me think through it step by step.

First, maybe in marketing. If a company wants to target specific customer segments more effectively, they could generate emails or ads tailored to what each group likes. So instead of random stuff that might not resonate with any audience, the AI creates something meaningful. That makes sense because customers might have unique preferences.

Then there's e-commerce. I know some companies use recommendation algorithms—like when you search for a product and see which ones are probably related. Generative AI could expand on that by creating personalized gift items or suggesting similar products based on styles. How would that work? Maybe by analyzing the user's last浏览内容, using patterns to suggest what looks good together.

In [31]:
Markdown(response.choices[0].message.content)

<think>
Okay, so I'm trying to understand the business applications of generative AI. From what I've heard before, generative AI is great at creating things like text, images, and even music. But how exactly can businesses use this? Let me think through it step by step.

First, maybe in marketing. If a company wants to target specific customer segments more effectively, they could generate emails or ads tailored to what each group likes. So instead of random stuff that might not resonate with any audience, the AI creates something meaningful. That makes sense because customers might have unique preferences.

Then there's e-commerce. I know some companies use recommendation algorithms—like when you search for a product and see which ones are probably related. Generative AI could expand on that by creating personalized gift items or suggesting similar products based on styles. How would that work? Maybe by analyzing the user's last浏览内容, using patterns to suggest what looks good together.

Another area could be content curation. Content creators might use this for free stuff—like generates can be used for art, tech parts, decorations. It adds value where there was nothing before. For example, a content creator in fashion who usually sells only new collections could maybe create promotional graphics to showcase old models online.

In sports, it's probably about creating content like videos or merchandise. People love seeing real stories behind what they watch. Companies could adapt existing videos into educational formats for free audiences. Maybe even live streaming content where AI knows how the audience will engage, making shows more vibrant and engaging automatically.

Personalized services are a big one too. For instance, personal loans might offer tailored advice based on someone's financial history. Generative AI can predict risks or possible issues without asking, which is confidential. It also can assess job opportunities through resumes—like suggesting roles someone could have in different industries even if the experience isn't listed.

Social media is another area I've seen companies trying out. Personalized content like posts on Tesla Cybertruck videos might reach many people quickly because it's engaging. Influencers could showcase real-world applications from around the world, which helps expand their reach.

Scheduling seems more traditional but can be modern with GAGA (genetic algorithm for assignment problems). This would optimize tasks by reducing costs or improving efficiency in scheduling jobs at the most cost-effective times or using resources better.

Healthcare and education must have ways to leverage this. In teaching, virtual reality could help students explore concepts visually without seeing 3D objects physically. For healthcare, generating medical images like X-rays from CT scans could offer free access for everyone or highlight issues earlier, helping doctors make better decisions.

Cannetworks might be in the works. AI-driven products that track agricultural plants remotely using satellite data, making it easier to manage crop yields and water resources without turning fertilizers into money. That's a big win because you save on fertilizer expenses.

Autonomous vehicles would benefit from real-time data showing future scenarios. When testing self-driving cars, they can predict what might happen around corners or traffic jams ahead—allowing them to adjust their paths slightly before reacting too much to sudden events.

In advertising, spotlighting content that directly aligns with user interests could drive higher engagement and more conversions. Instead of just clicking through ads randomly, AI personalizes which links users click on, making campaigns more effective by targeting exactly who they want to reach.

Healthcare dashboards could use medical imaging like CT scans but without the cost so patients can access important information better—potentially before it's too late for treatment or recovery. This helps doctors make more informed decisions.

Lastly, content creation plays a role in entertainment. Personal stories through videos add depth beyond just showing something; their impact is richer, affecting people on an emotional level that ads might not reach the same way.

Putting all of this together, I think generative AI has a wide range of applications across various industries. It's making products and services more relevant, engaging, and cost-effective for users. Companies are starting to see results by leveraging these capabilities, which makes businesses both innovative and efficient.
</think>

Generative AI is revolutionizing various industries by enhancing relevance, engagement, and efficiency. Here are some key business applications across multiple sectors:

1. **Marketing**:
   - Personalized marketing: Tailored customer-specific offers based on preferences.
   - Content recommendation: Predictive insights for relevance in online content submissions.

2. **E-commerce**:
   - Personalized gift items and recommendations using last浏览内容 patterns.
   - E-commerce recommendations enhancing shopping experience.

3. **Content Curation**:
   - Personalized gifts and art for new collections, offering value where there was nothing before.
   - Virtual reality videos on automotive subjects to reach millions quickly.

4. **Sports**:
   - Updated video content for educational use, leveraging real content without fees.
   - Live streaming optimizations with AI-driven engagement.

5. **Personalized Services**:
   - Financial advice tailored by credit history and security features.
   - Job placement insights based on resumes adjusted as they grow.

6. **Social Media**:
   - Personal posts to reach a wide audience instantly.
   - Influencer showcases from diverse regions aiding mass awareness.

7. **Scheduling**:
   - Optimal task scheduling using GAGA for cost and efficiency reduction.

8. **Healthcare and Education**:
   - Visual exploration of teaching concepts via VR simulations.
   - Medical imaging tools that enhance accessibility, saving costs in healthcare.

9. **Cannetworks**:
   - Real-time agricultural data tracking from satellites to reduce fertilizer expenses.

10. **Autonomous Vehicles**:
    - Predictive road scenarios during testing to guide safer navigation.

11. **Advertising**:
    - Spelling targeted clicks improving ROI and user engagement.

12. **Healthcare Dashboards**:
    - Access medical images without upfront costs, aiding in timely treatment and recovery.

13. **Content Creation**:
    - Rich, emotionally engaging content that resonates beyond ads on platforms like YouTube.

By leveraging generative AI, businesses are enhancing product relevance, engagement, and efficiency, driving innovation and cost savings across industries.